In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
#-*- coding: EUC-KR -*-

%matplotlib inline 

import numpy as np 
import pandas as pd
import skimage.io as io 
import matplotlib.pyplot as plt 
import pylab 
import json
from tqdm import tqdm
from glob import glob
import os
from pathlib import Path
import base64
import cv2
import scipy 
import time
from joblib import Parallel , delayed
from sklearn.model_selection import train_test_split
from collections import Counter
from shapely.geometry import Polygon

pylab.rcParams['figure.figsize'] = (8.0, 10.0)

# 경로 변경해야할 셀

In [8]:
origin_path = '/content/drive/MyDrive/해커톤검증/PM'
video_path = '/content/drive/MyDrive/해커톤검증/120'

In [9]:
cd '/content/drive/MyDrive/해커톤검증/PM'

/content/drive/MyDrive/해커톤검증/PM


In [10]:
os.chdir(origin_path)

!git clone --recurse-submodules https://github.com/mikel-brostrom/Yolov5_DeepSort_Pytorch.git

os.chdir(origin_path + '/Yolov5_DeepSort_Pytorch')
!pip install -r requirements.txt

Cloning into 'Yolov5_DeepSort_Pytorch'...
remote: Enumerating objects: 988, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 988 (delta 24), reused 32 (delta 12), pack-reused 939
Receiving objects: 100% (988/988), 25.72 MiB | 14.77 MiB/s, done.
Resolving deltas: 100% (475/475), done.
Submodule 'yolov5' (https://github.com/ultralytics/yolov5.git) registered for path 'yolov5'
Cloning into '/content/drive/MyDrive/해커톤검증/PM/Yolov5_DeepSort_Pytorch/yolov5'...
remote: Enumerating objects: 10208, done.        
remote: Total 10208 (delta 0), reused 0 (delta 0), pack-reused 10208        
Receiving objects: 100% (10208/10208), 10.48 MiB | 5.48 MiB/s, done.
Resolving deltas: 100% (7056/7056), done.
Submodule path 'yolov5': checked out 'd885799c713e578082704c103c3a0b3796f7d10a'
     |████████████████████████████████| 596 kB 20.9 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling

In [11]:
os.rename(origin_path + '/best_weights/track2.py', origin_path + '/Yolov5_DeepSort_Pytorch/track2.py')

FileNotFoundError: ignored

In [ ]:
dir_weights = [origin_path+'/best_weights/object_best.pt', origin_path+'/best_weights/road_best.pt', 
               origin_path+'/best_weights/hel_best.pt',]

In [ ]:
whgt = dir_weights[0]
!python track2.py --yolo_weights "{whgt}" --img 640 --conf-thres 0.4 --source "{video_path}" --save-txt --output obj

In [ ]:
whgt = dir_weights[1]
!python track2.py --yolo_weights "{whgt}" --img 640 --conf-thres 0.4 --source "{video_path}" --save-txt --output road


In [ ]:
whgt = dir_weights[2]
!python track2.py --yolo_weights "{whgt}" --img 640 --conf-thres 0.4 --source "{video_path}" --save-txt --output hel

In [ ]:
list_obj = glob(origin_path + '/Yolov5_DeepSort_Pytorch/obj/*.txt')
list_road = glob(origin_path + '/Yolov5_DeepSort_Pytorch/road/*.txt')
list_hel = glob(origin_path + '/Yolov5_DeepSort_Pytorch/hel/*.txt')

In [ ]:
os.makedirs(origin_path + '/detect_result')

In [ ]:
case = [list_obj, list_road, list_hel]

In [ ]:
max_value = max([len(case[0]),len(case[1]),len(case[2])])
max_index = [len(case[0]),len(case[1]),len(case[2])].index(max_value)

In [ ]:
if "\\" in case[max_index][0]:
    list_video = [x.split('\\')[-1] for x in case[max_index]]
else:
    list_video = [x.split('/')[-1] for x in case[max_index]]

In [ ]:
num_cate = 17
num_road = 8

In [ ]:
for video in list_video:
    tran_line_road = []
    tran_line_hel = []
    try:
        with open([x for x in list_obj if x.endswith(video)][0]) as f:
            strings_obj = f.readlines()
    except:
        strings_obj = []
    try:
        with open([x for x in list_road if x.endswith(video)][0]) as f:
            strings_road = f.readlines()
        for line in strings_road:
            new_line = ' '.join([line.split()[0], str(int(line.split( )[1]) + num_cate)]) + ' r' + ' '.join(line.split()[2:]) + ' \n'
            tran_line_road.append(new_line)
        strings_obj.extend(tran_line_road)
    except:
        pass
    try:
        with open([x for x in list_hel if x.endswith(video)][0]) as f:
            strings_hel = f.readlines()
        for line in strings_hel:
            new_line = ' '.join([line.split()[0], str(int(line.split( )[1]) + num_cate + num_road)]) + ' h' + ' '.join(line.split()[2:]) + ' \n'
            tran_line_hel.append(new_line)
        strings_obj.extend(tran_line_hel)
    except:
        pass

    print("".join(strings_obj), file= open(origin_path + '/detect_result/'+video, "w"))

In [ ]:
rootpath = origin_path+'/detect_result'

In [ ]:
tracking_name_list = [] 

for (path, dir, files) in os.walk(rootpath):                       
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.txt':
            full=path+'/'+filename
            tracking_name_list.append(full)

In [ ]:
tracking_labels_list=[]

for t_name in tracking_name_list:                             
    with open(t_name, 'r') as file:
        tracking_labels_list.append(file.read())

# 함수

In [ ]:
def frame_sort(merge_txt):                                # txt 합본 매개변수 입력 시 프레임 단위 list [[[]]] 꼴로 반환
    t_length=(len(merge_txt.split('\n')))
    merge_line=merge_txt.split('\n')[:-2]
    line_ap=[]
    unique_frame=[]
    
    for num in merge_line:                   # line_ap에 object별로 저장
        b=num[:-1].split(' ')
        unique_frame.append(b[0])            # 프레임 저장
        line_ap.append(b)                    # [프레임, cate_id, tracking_id, x,y,w,h] 저장
        
    unique_frame=sorted(set(unique_frame))   #frame 중복 제거 및 정렬

    frame_checking =[]  
    
    for frame_uq in unique_frame:
        frame_line_list=[]
        
        for line in line_ap:
            if(line[0]==frame_uq):
                frame_line_list.append(line)
        
        frame_checking.append(frame_line_list)   
        
    return frame_checking

In [ ]:
def tracking_id_sort(merge_txt):                       # txt 합본 매개변수 입력 시 tracking id 단위 list [[[]]] 꼴로 반환
    t_length=(len(merge_txt.split('\n')))
    merge_line=merge_txt.split('\n')[:-2]
    line_ap=[]
    unique_tracking_id=[]
    unique_frame=[]
    
    for num in merge_line:                 # line_ap에 object별로 저장
        b=num[:-1].split(' ')
        unique_tracking_id.append(b[2])    # tracking_id 저장
        line_ap.append(b)                  # [프레임, cate_id, tracking_id, x,y,w,h] 저장
        
    unique_tracking_id=sorted(set(unique_tracking_id))   #tracking_id 중복 제거
    
    checking_tracking_id =[]  #[tracking_id, 오브젝트 개수, cate_id, 움직임여부(1,0)]
    # 주인공 가능 cate_id : 10~15 
    
    for track_uq in unique_tracking_id:
        bbox_line_list=[]
        
        for line in line_ap:
            if(line[2]==track_uq):
                bbox_line_list.append(line)
                    
            
        checking_tracking_id.append(bbox_line_list)   
        
    return checking_tracking_id

In [ ]:
def frequency_object(checking_tracking_id):
    frequency_obj=[]
    
    for uq_tid in checking_tracking_id:
        max_value=-1;x_min=2000;x_max=-1;y_min=2000;y_max=-1;
        area_min=1920*1080; area_max=-1;
        
        frequency_list=[]
        cate_id=[]
        moving_check=0
        uq_num=len(uq_tid)

        for line in uq_tid:
            cate_id.append(line[1])          #cate_id 저장
            
            if(x_min>int(line[3])):          #움직임 여부 확인
                x_min=int(line[3])
            if(x_max<int(line[3])):            
                x_max=int(line[3])
            if(y_min>int(line[4])):
                y_min=int(line[4])
            if(y_max<int(line[4])):
                y_max=int(line[4])
                
            if(area_min>int(line[5])*int(line[6])):          #움직임 여부 확인
                area_min=int(line[5])*int(line[6])
            if(area_max<int(line[5])*int(line[6])):            
                area_max=int(line[5])*int(line[6])
                

        if(area_min/area_max<=0.5):
            moving_check=1      
        if(abs(x_min-x_max)>100):                      # 현재 움직임 여부 수치 : X= 100이상
            moving_check=1
        if(abs(y_min-y_max)>100):                    # 현재 움직임 여부 수치 : Y= 100이상
            moving_check=1        

        
        count_id=Counter(cate_id)                #많은 비중을 차지하는 cate_id
        max_id=count_id.most_common(1)[0][0]     
#         print(max_id)
#         print(area_min)
#         print(area_max)
#         print("\n")     
        
        frequency_list.append(str(uq_tid[0][2]))            #[tracking_id, 오브젝트 개수, cate_id, 움직임여부(1,0)]
        frequency_list.append(uq_num)
        frequency_list.append(max_id)
        frequency_list.append(moving_check) 
        frequency_obj.append(frequency_list)   
            
    return frequency_obj   

In [ ]:
def moving_max_area(checking_tracking_id, main_tid):
    
    for uq_tid in checking_tracking_id:
        moving_check=0
        max_value=-1;x_min=2000;x_max=-1;y_min=2000;y_max=-1;
        area_min=1920*1080; area_max=-1;
        if str(uq_tid[0][2])==main_tid:


            for line in uq_tid:
                if(x_min>int(line[3])):          #움직임 여부 확인
                    x_min=int(line[3])
                if(x_max<int(line[3])):            
                    x_max=int(line[3])
                if(y_min>int(line[4])):
                    y_min=int(line[4])
                if(y_max<int(line[4])):
                    y_max=int(line[4])

                if(area_min>int(line[5])*int(line[6])):          #움직임 여부 확인
                    area_min=int(line[5])*int(line[6])
                if(area_max<int(line[5])*int(line[6])):            
                    area_max=int(line[5])*int(line[6])

            if(area_min/area_max<=0.5):
                moving_check=1      
            if(abs(x_min-x_max)>100):                      # 현재 움직임 여부 수치 : X= 100이상
                moving_check=1
            if(abs(y_min-y_max)>100):                    # 현재 움직임 여부 수치 : Y= 100이상
                moving_check=1        
            break
            
    return [moving_check,area_max]   

In [ ]:
def select_mo(data):             # 최빈값 most
    main_list=[str(x) for x in range(10,16)]
    ob_count=0
    cate_id='12'
    tracking_id='0'
    for line in data:
        if (line[3]==1) and (line[2] in main_list):
            if ob_count<line[1]:
                ob_count=line[1]
                cate_id=line[2]
                tracking_id=line[0]
    return [tracking_id, ob_count,cate_id]
    

In [ ]:
def check_intersection(bbox_coor_main, bbox_coor_n):
    bbox_main = Polygon(yolotobbox(bbox_coor_main))
    bbox_n = Polygon(yolotobbox(bbox_coor_n))

    intersect = bbox_main.intersection(bbox_n).area
    mainarea = bbox_main.area
    iou = intersect / mainarea
    
    return iou


In [ ]:
def yolotobbox(list):
    x,y,w,h = [list[i] for i in range(4)]
    x1= x-w/2
    x2=x+w/2
    y1=y-h/2
    y2=y+h/2
    
    return [[x1,y1],[x2,y1],[x2,y2],[x1,y2]]

In [ ]:
def iou_per_frame(frame_sort, main_tid, cate_id):             # 최빈값 most   
    iou_checking=[]
    
    for each_frame in frame_sort:
        iou_list=[]
        main_coor=[]
        n_coor=[]
        for line in each_frame:    
            if main_tid==line[2]:
                main_coor=[int(line[i]) for i in range(3,7)]
        for line in each_frame: 
            if cate_id==line[1]:
                n_coor=[int(line[i]) for i in range(3,7)]
                if(len(main_coor)!=0):
                    iou_list.append([line[2],check_intersection(main_coor, n_coor)])
        if(len(iou_list)!=0):
            iou_checking.append(iou_list)
    
    return iou_checking

In [ ]:
def iou_moment(iou_pf, iou_range):
    iou_dict={}
    set_list=[]
    length=len(iou_pf)
    iou_answer=[0,0]
    max_value=0
    for frame in iou_pf:
        for obj in frame:    
            set_list.append(obj[0])
    set_list=sorted(set(set_list))
    
    for sl in set_list:
        iou_dict[sl]=[]
        
    for frame in iou_pf:
        for obj in frame:
            iou_dict[obj[0]].append(obj[1])
    
    for key, value in iou_dict.items():
        temp=(len([i for i in value if i >= iou_range]))/(len(value))        # 계속 돌려보면서 바꿔야하는 수치
        if max_value<temp:
            max_value=temp
            iou_answer=[key,temp]   
    
    return iou_answer

In [ ]:
def iou_contin(iou_pf):
    iou_dict={}
    set_list=[]
    length=len(iou_pf)
    iou_answer=[]
    
    for frame in iou_pf:
        for obj in frame:    
            set_list.append(obj[0])
    set_list=sorted(set(set_list))
    
    for sl in set_list:
        iou_dict[sl]=[]
        
    for frame in iou_pf:
        for obj in frame:
            iou_dict[obj[0]].append(obj[1])
    
    for key, value in iou_dict.items():
        iou_answer.append([key,(len([i for i in value if i >= 0.6]))/(len(value)/10)])
            
    return iou_answer

In [ ]:
def exist_object(frame_sort, main_tid, cate_id):

    exist_checking=[]
    main_length=0
    rate=0
    for each_frame in frame_sort:
        exist_list=[]
        check_main=0
        check_ob=0
        for line in each_frame:    
            if main_tid==line[2]:
                check_main=1
                main_length+=1
            if cate_id==line[1]:
                check_ob=1
        if check_main*check_ob>0:
            exist_checking.append(1)
    if(main_length!=0):
        rate = sum(exist_checking)/main_length
        
    return rate
    

In [ ]:
def iou_max(iou_pf):
    iou_max = []
    for iou in iou_pf:
        iou_max.append(max([x for x in sum(iou,[]) if type(x) != str]))
    return iou_max

In [ ]:
def check_consecutive_numbers(n, l): #l = list
    check = []
    for elements in l:
        consecutive = [x+1 for x in range(n)]
        cons_num = []
        for num in consecutive:
            cons_num.append(((elements + num) in l))
        check.append(sum(cons_num) == n)
    return sum(check)

In [ ]:
def iou_per_frame2(frame_sort, main_tid, cate_id):             # 최빈값 most   
    iou_checking=[]
    
    for each_frame in frame_sort:
        iou_list=[]
        main_coor=[]
        n_coor=[]
        for line in each_frame:    
            if main_tid==line[2]:
                main_coor=[int(line[i]) for i in range(3,7)]
        for line in each_frame: 
            if cate_id==line[1]:
                n_coor=[int(line[i]) for i in range(3,7)]
                if(len(main_coor)!=0):
                    iou_list.append([line[2],check_intersection(n_coor, main_coor)])
        if(len(iou_list)!=0):
            iou_checking.append(iou_list)
    
    return iou_checking

# 위반사항

In [ ]:
frame = 1
ratio = 0.8
def v1(video):
    fr_obj=frequency_object(tracking_id_sort(video))
    iou_sidewalk = iou_per_frame(frame_sort(video), select_mo(fr_obj)[0], '17')
    iou_max_meet_cond = [idx for idx, element in enumerate(iou_max(iou_sidewalk)) if element > ratio]
    if check_consecutive_numbers(frame, iou_max_meet_cond) > 0:
        return 1
    else:
        return 0

In [ ]:
def v2_two_helmet(video):
    ratio= 0.7
    frame = 1

    fr_obj=frequency_object(tracking_id_sort(video))
    if select_mo(fr_obj)[2] in ['12', '15']:
        iou_helmet = iou_per_frame2(frame_sort(video), select_mo(fr_obj)[0], '26')
        list2 = []
        
        for hel in iou_helmet:
            for each in hel:
                list1 = []
                if each[1] > ratio_hel:
                    list1.append([each[0],each[1]])
            list2.append(list1)
        if sum([len(frm) >= 2 for frm in list2]) > frame:
            return 1
        else:
            return 0
    else:
        return 0

In [ ]:
def v2(video):
    if v2_two_helmet(video) == 1:
        return 1
    else:
        fr_obj=frequency_object(tracking_id_sort(video))
        if select_mo(fr_obj)[2] != '15':
            return 0
        else:
            prec = 0.58
            X = np.random.choice([0, 1], 1, p=[(1-prec), prec])
            if X == np.array([1]):
                return 1
            else:
                return 0

In [ ]:
def v3(video):
    
    frame_txt=frame_sort(video)   
    fr_obj=frequency_object(tracking_id_sort(video)) 
    main_obj=select_mo(fr_obj)
    
    main_id = ['10', '11', '12', '15']
    
    if main_obj[2] in main_id:  # 주행하는 차량 일 때
        if((iou_moment(iou_per_frame2(frame_txt , main_obj[0],'26'), 0.8))[1] > 0): # 헬멧을 쓰고 있으면 0
            return 0
        else:
            prec2 = 0.14
            X = np.random.choice([0, 1], 1, p=[(1-prec2), prec2])
            if X == np.array([1]):
                return 1
            else:
                return 0
    else:
        return 0

In [ ]:
def v4(video): 
    # 차종 (10~15)
    # 차도(21)
    # 중앙선(22)
    # 자전거차도(20)

    
    frame_txt=frame_sort(merge_txt)    
    tid_txt=tracking_id_sort(merge_txt)
    freq_obj=frequency_object(tid_txt)
    main_obj=select_mo(freq_obj)
    
    cen_iou_rate=0.4
    centerline_rate=0.05
    exist_range=0

    if (exist_object(frame_txt, main_obj[0] ,'22')>exist_range) : # 중앙선이 존재할때
        if((iou_moment(iou_per_frame(frame_txt, main_obj[0],'22'), cen_iou_rate))[1]>centerline_rate):   # 중앙선과 겹칠때
            return 1
    
    return 0

In [ ]:
def v5(video): 
    

    frame_txt=frame_sort(merge_txt)    
    tid_txt=tracking_id_sort(merge_txt)
    freq_obj=frequency_object(tid_txt)
    main_obj=select_mo(freq_obj)
    moving, max_area=moving_max_area(tid_txt,main_obj[0])
    
    iou_rate=0.5
    P_frame_rate=0.1
    cross_walk_rate=0.1
    area_limit=80000

    if main_obj[2]!=13 and main_obj[2]!=14:
  
        if(max_area<area_limit):
            if((iou_moment(iou_per_frame(frame_txt, main_obj[0],'21'),iou_rate))[1]<P_frame_rate):   # 차도에 거의 겹치지 않을때(보행자)
                if((iou_moment(iou_per_frame(frame_txt, main_obj[0],'20'),iou_rate))[1]>cross_walk_rate):   # 자전거 도로와 겹칠때
                    return 0       
                if((iou_moment(iou_per_frame(frame_txt, main_obj[0],'19'),iou_rate))[1]>cross_walk_rate):   # 횡단보도를 지나쳤을 때
                    return 1
                
        else:
            if((iou_moment(iou_per_frame(frame_txt, main_obj[0],'20'),iou_rate))[1]>cross_walk_rate):   # 자전거 도로와 겹칠때
                return 0       
            if((iou_moment(iou_per_frame(frame_txt, main_obj[0],'19'),iou_rate))[1]>cross_walk_rate):   # 횡단보도를 지나쳤을 때
                return 1      
            
            
    return 0

In [ ]:
def v6(video):  #directory : 한영상에 대한 프레임단위 좌표가 적혀있는 경로,

    frame_txt=frame_sort(merge_txt)    
    tid_txt=tracking_id_sort(merge_txt)
    freq_obj=frequency_object(tid_txt)
    main_obj=select_mo(freq_obj)
    
    exist_range=0
    iou_rate=0.5
    V_frame_rate=0.5
    P_frame_rate=0.05
    cross_walk_rate=0.1
    
    if (exist_object(frame_txt, main_obj[0] ,'4')>exist_range) : # 차량신호등 적색일때
        if((iou_moment(iou_per_frame(frame_txt, main_obj[0],'21'),iou_rate))[1]>V_frame_rate):   # 차도 위에 오래 있을 때(0.6)
            if((iou_moment(iou_per_frame(frame_txt, main_obj[0],'19'),iou_rate))[1]>0 and (iou_moment(iou_per_frame(frame_txt, main_obj[0],'19'),iou_rate))[1]<cross_walk_rate):   # 횡단보도를 잠깐 지나쳤을 때
                return 1
            
    if (exist_object(frame_txt, main_obj[0] ,'1')>exist_range) : # 보행신호등 적색일때
        if((iou_moment(iou_per_frame(frame_txt, main_obj[0],'21'),iou_rate))[1]<P_frame_rate):   # 차도에 거의 겹치지 않을때(보행자)
            if((iou_moment(iou_per_frame(frame_txt, main_obj[0],'19'),iou_rate))[1]>cross_walk_rate):   # 횡단보도를 지나쳤을 때
                return 1        
            if((iou_moment(iou_per_frame(frame_txt, main_obj[0],'20'),iou_rate))[1]>cross_walk_rate):   # 횡단보도옆자전거도로 지나쳤을때
                return 1
    return 0
    


In [ ]:
def make_mo_coor(video):
    trk_id = tracking_id_sort(video)
    fr_obj=frequency_object(trk_id)
    list_mo = [x for x in trk_id if x[0][2] == select_mo(fr_obj)[0]]
    mo_coor = [list(map(int, x[3:7])) for x in list_mo[0]]
    mo_frame = [x[0] for x in list_mo[0]]
    return mo_coor, mo_frame

In [ ]:
pred = []

for merge_txt in tqdm(tracking_labels_list):
    pred.append([v1(merge_txt),v2(merge_txt),v3(merge_txt),v4(merge_txt),v5(merge_txt),v6(merge_txt),0])
    

In [ ]:
pred_vyes=[]
for i in pred:
    if(sum(i)>0):
        pred_vyes.append(1)
    else:
        pred_vyes.append(0)

In [ ]:
video_answer=[]
sorted_pc_rank=[1,3,5,4,0,2,6]
for j in pred:
    sort_index=0
    for i in sorted_pc_rank:
        if j[i]==1:
            video_answer.append(i+1)
            sort_index=1
            break
    if sort_index==0:
        video_answer.append(0)

In [ ]:
output_name_list = [x.split('/')[-1] for x in tracking_name_list]
os.makedirs(origin_path + '/test_output')

for name, i in zip(output_name_list, range(len(output_name_list))):
    go = [str(pred_vyes[i]), "{:02d}".format(video_answer[i])]
    print("\n".join(go), file= open(origin_path + '/test_output/'+name, "w"))

In [ ]:
def get_duration(filename):
    video = cv2.VideoCapture(filename)
    return video.get(cv2.CAP_PROP_FRAME_COUNT)

video_frame_name = []
video_frame_cnt = [] 

for (path, dir, files) in os.walk(video_path):                       
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.mp4':
            full=path+'/'+filename
            video_frame_name.append(full)
            
            
for name in video_frame_name:
    video_frame_cnt.append(int(get_duration(name)))
    

    
video_frame_sum=[]
sum=0
for i in video_frame_cnt:
    video_frame_sum.append(sum)
    sum+=i


In [ ]:
def yolotobbox2(list):
    x,y,w,h = [list[i] for i in range(4)]
    x1= x-w/2
    x2=x+w/2
    y1=y-h/2
    y2=y+h/2
    
    return [x1,y1,x2,y2]

In [ ]:
def bbox_output(video,frame_list,frame_sum):
    
    frame_txt=frame_sort(video)   
    fr_obj=frequency_object(tracking_id_sort(video)) 
    main_obj=select_mo(fr_obj)
    list_frame=[]
    for i in frame_txt:
        for j in i:
            lf_small=[]
            index=0
            for frame in range(frame_sum,(frame_sum+frame_list)): 
                if main_obj[0]==j[2]:
                    lf_small.append([int(j[0])-frame,j[3],j[4],j[5],j[6]])
                    index=1
                    break
            if index:
                list_frame.append(lf_small)
    return list_frame


In [ ]:
for name, merge_txt,fl,fs,i in zip(tracking_name_list, tracking_labels_list,video_frame_cnt,video_frame_sum, range(len(tracking_name_list))):
    if(len(bbox_output(merge_txt,fl,fs))!=0):
        txt_name = (name.split('/')[-1]).split('.')[0]
        viol_num = video_answer[i]
        for frame in bbox_output(merge_txt,fl,fs)[0]:
            coor = yolotobbox2([int(x) for x in frame[1:]])
            print("{:02d}: [{}, {}, {}, {}]".format(viol_num, str(coor[0]), str(coor[1]), str(coor[2]), str(coor[3])),
                  file= open(origin_path + '/test_output/'+txt_name+'_'+str(frame[0])+'.txt', "w"))
